# Imports

In [ ]:
# install PyArabic library for Arabic preprocessing
!pip install PyArabic

     |████████████████████████████████| 126 kB 5.1 MB/s 


In [ ]:
# install tensorflow_addons for AdamW optimizer
!pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 5.1 MB/s 


In [ ]:
# Standard libraries imports
import io
import re
import ast
import math
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
from lxml import etree
import pyarabic.araby as araby

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import layers

# Hardware Detection

In [ ]:
# Detect hardware
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
    tpu = None
    gpus = tf.config.experimental.list_logical_devices("GPU")
    
# Select an appropriate distribution strategy
if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu) # Going back and forth between TPU and host is expensive. Better to run 128 batches on the TPU before reporting back.
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])  
elif len(gpus) > 1:
    strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
    print('Running on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
    strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
    print('Running on single GPU ', gpus[0].name)
else:
    strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
    print('Running on CPU')
print("Number of accelerators: ", strategy.num_replicas_in_sync)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.3.48.146:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.3.48.146:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Running on TPU  ['10.3.48.146:8470']
Number of accelerators:  8


# Configs

In [ ]:
# configurations class
class config:

  MAXLEN = 17 # maximum length of sequence

  # model configs
  BATCH_SIZE_PER_REPLICA = 256 # batch size per replica
  BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync # global batch size
  EPOCHS = 100 # number of epochs
  LEARNING_RATE = 1E-4 # learning rate
  BUFFER = 1024 # buffer size for shuffling the training set
  EMBEDDING_SIZE = 256 # embedding size
  NUM_HEADS = 8 # number of heads
  NUM_BLOCKS = 4 # number of encoder blocks
  KEY_DIM = 64 # dimesion of the key, value, and query
  DROPOUT = 0.0 # dopout rate
  SKIP_RATIO = 0.0 # skip connection ratio

# Dataset

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# list of all Arabic characters
chars = list('اٱبتةثجحخدذرزسشصضطظعغفقكلمنهويءآأؤإئىی')
print('Number of Arabic Characters:', len(chars))

# character to index dictionary
char_to_index = dict((char, index+3) for (index, char) in enumerate(chars))
# index to character dictionary
index_to_char=  dict((index+3, char) for (index, char) in enumerate(chars))

char_to_index['p'] = 0 # pad
char_to_index['s'] = 1 # separator
char_to_index['_'] = 2 # mask


index_to_char[0] = 'p' # pad
index_to_char[1] = 's' # separator
index_to_char[2] = '_' # mask

# additional char_to_index and index_to_char extracted manually as shown in next 
# cells. They were extracted and then saved into a CSV file
df = pd.read_csv('https://drive.google.com/uc?export=download&id=1lQFU7XFy82-1dE5kPK7yDLpJ6tleycCC')
chars_2 = df['char'].values.tolist()
indexes_2 = df['value'].values.tolist()
del df

# additional character to index dictionary
char_to_index_2 = dict(zip(chars_2, indexes_2))
# additional index to character dictionary
index_to_char_2 = dict(zip(indexes_2, chars_2))

Number of Arabic Characters: 38


In [ ]:
# read the prepared pre-trainig datasets
train_df = pd.read_csv('https://drive.google.com/uc?export=download&id=1zu7Ohjgsodw3ffJVpAHDZfj2UIWT39QE')
valid_df = pd.read_csv('https://drive.google.com/uc?export=download&id=1-1RggfVygU5svgMeZjgfK1Su_-AXZ_6V')

In [ ]:
# create lists to fill with the indexes
# parsed from the datasaets
train_masked_labels, train_true_labels, train_masks = [], [], []
valid_masked_labels, valid_true_labels, valid_masks = [], [], []

# iterate over the training dataset and fill the three 
# lists with the true indexes, masked indexes, and masks
for i in tqdm(range(len(train_df))):
  train_masked_labels.append(ast.literal_eval(train_df.iloc[i, 0]))
  train_true_labels.append(ast.literal_eval(train_df.iloc[i, 1]))
  train_masks.append(ast.literal_eval(train_df.iloc[i, 2]))

# iterate over the validation dataset and fill the three 
# lists with the true indexes, masked indexes, and masks
for i in tqdm(range(len(valid_df))):
  valid_masked_labels.append(ast.literal_eval(valid_df.iloc[i, 0]))
  valid_true_labels.append(ast.literal_eval(valid_df.iloc[i, 1]))
  valid_masks.append(ast.literal_eval(valid_df.iloc[i, 2]))

# delete the training and validation sets and keep only the lists
del train_df, valid_df

100%|██████████| 1242/1242 [00:00<00:00, 4521.76it/s]


In [ ]:
# create a TensorFlow training dataset that performs multiple functions including
# shuffling, batching, and prefetching
train_ds = tf.data.Dataset.from_tensor_slices((tf.constant(train_masked_labels), tf.constant(train_true_labels), tf.constant(train_masks)))
train_ds = train_ds.shuffle(config.BUFFER).batch(config.BATCH_SIZE, drop_remainder=False).prefetch(-1)

# create a TensorFlow validation dataset that performs multiple functions including
# batching and prefetching
valid_ds = tf.data.Dataset.from_tensor_slices((tf.constant(valid_masked_labels), tf.constant(valid_true_labels), tf.constant(valid_masks)))
valid_ds = valid_ds.batch(config.BATCH_SIZE, drop_remainder=False).prefetch(-1)

# Models

## Transformer

Use the TensorFlow functional API to develop the CBERT model.

In [ ]:
def encoder_block(inputs, num_heads=8, key_dim=64, dropout=0.0, linear_shape=128):
  # multi-head attention layer
  x = layers.MultiHeadAttention(num_heads=num_heads,
                                  key_dim=key_dim,
                                  dropout=dropout)(inputs, inputs)
  # add & norm
  x = layers.Add()([inputs, x])
  x1 = layers.LayerNormalization()(x)
  # feed-forward
  x = layers.Dense(linear_shape, activation='gelu')(x1)
  x2 = layers.Dense(inputs.shape[-1], activation='linear')(x)
  # add & norm
  x = layers.Add()([x1, x2])
  x = layers.LayerNormalization()(x)
  return x

In [ ]:
def encoder_model(inputs, num_blocks=4, num_heads=8, key_dim=64, dropout=0.0, skip_ratio=0.0):
  
  # embedding layer that maps each input character index into a vector of the specified length
  # in the config class
  embedding_layer = layers.Embedding(input_dim=len(chars)+3, output_dim=config.EMBEDDING_SIZE, input_length=config.MAXLEN, mask_zero=True)
  # use another trainable embedding layer to learn positional encoding 
  pos_encoding = layers.Embedding(input_dim=config.MAXLEN+1, output_dim=config.EMBEDDING_SIZE, input_length=config.MAXLEN, mask_zero=False)

  # embedding layer for characters
  embed = embedding_layer(inputs)
  # trainable positional encoding
  pos_encod = pos_encoding(tf.range(config.MAXLEN))
  # add pos_encod to embeddings
  encoded_embed = embed + pos_encod
  # multi-head attention
  mha = encoder_block(encoded_embed, num_heads=num_heads, key_dim=key_dim, dropout=dropout)
  mha = skip_ratio * encoded_embed + (1 - skip_ratio) * mha # skip connection to preserve signal
  # repeated blocks
  block_outputs = dict()
  for i in range(1, num_blocks):
    x = mha
    block_outputs[f'x{i}'] = x
    mha = encoder_block(encoded_embed, num_heads=num_heads, key_dim=key_dim, dropout=dropout, linear_shape=128)
    mha = skip_ratio * x + (1 - skip_ratio) * mha # skip connection to preserve signal
  block_outputs[f'x{i+1}'] = mha
  return block_outputs

In [ ]:
def cbert_model():
  # the full CBERT model
  inputs = layers.Input(shape=(config.MAXLEN,), name='cbert_input')
  bert_output = encoder_model(inputs, num_blocks=config.NUM_BLOCKS, 
                              num_heads=config.NUM_HEADS, key_dim=config.KEY_DIM,
                              dropout=config.DROPOUT, skip_ratio=config.SKIP_RATIO)
  final_encoder_name = [*bert_output][-1]
  dense = layers.Dense(len(chars)+3, activation='softmax')(bert_output[final_encoder_name])

  model = tf.keras.Model(inputs=inputs, outputs=dense)
  return model

In [ ]:
# instantiate the model and print the summary
cbert = cbert_model()
cbert.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 cbert_input (InputLayer)       [(None, 17)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 17, 256)      10496       ['cbert_input[0][0]']            
                                                                                                  
 tf.__operators__.add (TFOpLamb  (None, 17, 256)     0           ['embedding[0][0]']              
 da)                                                                                              
                                                                                                  
 multi_head_attention (MultiHea  (None, 17, 256)     526080      ['tf.__operators__.add[0][0]'

## LSTM

We investigate the performance of LSTM layers in this task

In [ ]:
def lstm_model():
  # LSTM-based model with straightforward connections
  inputs = layers.Input(shape=(config.MAXLEN,), name='cbert_input')
  embed = layers.Embedding(input_dim=len(chars)+3, output_dim=config.EMBEDDING_SIZE, input_length=config.MAXLEN, mask_zero=True)(inputs)
  lstm1 = layers.Bidirectional(layers.LSTM(256, return_sequences=True))(embed)
  lstm2 = layers.Bidirectional(layers.LSTM(256, return_sequences=True))(lstm1)
  lstm3 = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(lstm2)
  lstm4 = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(lstm3)
  output = layers.Dense(len(chars)+3, activation='softmax')(lstm4)
  model = tf.keras.Model(inputs=inputs, outputs=output)
  return model

In [ ]:
# instantiate the model and print the summary
model = lstm_model()
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cbert_input (InputLayer)    [(None, 17)]              0         
                                                                 
 embedding_2 (Embedding)     (None, 17, 256)           10496     
                                                                 
 bidirectional (Bidirectiona  (None, 17, 512)          1050624   
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 17, 512)          1574912   
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 17, 256)          656384    
 nal)                                                            
                                                           

# Custom Training

## TPU

## Transformer

In [ ]:
# 8 heads 4 blocks Adam optimizer

# clear the session to release all previous states generated by Keras
tf.keras.backend.clear_session()
# instantiate and compile the model inside the strategy scope defined previous
with strategy.scope():
  model = cbert_model()
  # use Sparse Categorical Cross-entropy as a loss function as labels are not one-hot encoded
  # use AdamW optimizer initially with a weight decay of 0.0
  # use the Sparse Categorical Accuracy metric to monitor and save the best model weights
  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                optimizer=tfa.optimizers.AdamW(learning_rate=config.LEARNING_RATE, weight_decay=0.0),
                metrics=tf.keras.metrics.SparseCategoricalAccuracy())

# initialize the best accuracy at - infinity
best_acc = -np.inf
# define number of steps per training epoch
train_steps = (len(train_masked_labels)//config.BATCH_SIZE) + 1
# define number of steps per evaluation epoch
valid_steps = (len(valid_masked_labels)//config.BATCH_SIZE) + 1

# for each epoch iterate over the whole training and validation sets
for epoch in range(config.EPOCHS):
  print(f'Epoch: {epoch+1}/{config.EPOCHS}')
  
  # initialize losses and accuracies list
  losses = []
  accs = []
  # iterate over the training dataset
  for step, (x, y, mask) in enumerate(tqdm(train_ds, total=train_steps)):
    # stop when hitting the number of steps per epoch
    if step == train_steps:
      break
    # train on each batch and pass the sample weights which gives weights to
    # the masked tokens only
    train_results = model.train_on_batch(x=x, y=y, sample_weight=mask,
                                         reset_metrics=True, return_dict=True)
    # append the loss value 
    losses.append(train_results['loss'])
    # append the accuracy value
    accs.append(train_results['sparse_categorical_accuracy'])
    
  print('Train loss:', np.mean(losses), 'Train accuracy', np.mean(accs))
  
  # initialize losses and accuracies list
  losses = []
  accs = []
  # iterate over the validation dataset
  for step, (x, y, mask) in enumerate(tqdm(valid_ds, total=valid_steps)):
    # stop when hitting the number of steps per epoch
    if step == valid_steps:
      break
    # evaluate on each batch and pass the sample weights which gives weights to
    # the masked tokens only
    valid_results = model.test_on_batch(x=x, y=y, sample_weight=mask,
                                         reset_metrics=True, return_dict=True)
    # append the loss value 
    losses.append(valid_results['loss'])
    # append the accuracy value
    accs.append(valid_results['sparse_categorical_accuracy'])
  # monitor the accuracy metric and save model weights whenever there is an improvement
  if np.mean(accs) > best_acc:
    best_acc = np.mean(accs)
    model.save_weights('best_pre_trained_weights.h5')
    print('new model weights saved')
    print('accuracy improved to', best_acc)
  else:
    print("accuracy didn't improve from", best_acc)
  print('Valid loss:', np.mean(losses), 'Valid accuracy', np.mean(accs))

Epoch: 1/100


100%|██████████| 6/6 [00:35<00:00,  5.91s/it]


Train loss: 0.9946330984433492 Train accuracy 0.05352939541141192


100%|██████████| 1/1 [00:05<00:00,  5.01s/it]


new model weights saved
accuracy improved to 0.1101638674736023
Valid loss: 0.9218062162399292 Valid accuracy 0.1101638674736023
Epoch: 2/100


...

accuracy didn't improve from 0.8211612701416016
Valid loss: 0.5895759463310242 Valid accuracy 0.8194562196731567
Epoch: 100/100


100%|██████████| 6/6 [00:03<00:00,  1.53it/s]


Train loss: 0.5744477013746897 Train accuracy 0.8179977536201477


100%|██████████| 1/1 [00:00<00:00,  1.55it/s]

accuracy didn't improve from 0.8211612701416016
Valid loss: 0.5895949006080627 Valid accuracy 0.820214033126831


In [ ]:
# !mv /content/best_pre_trained_weights.h5 /content/drive/MyDrive/NLP_Course

## LSTM

In [ ]:
# LSTM Adam optimizer

# clear the session to release all previous states generated by Keras
tf.keras.backend.clear_session()
# instantiate and compile the model inside the strategy scope defined previous
with strategy.scope():
  model = lstm_model()
  # use Sparse Categorical Cross-entropy as a loss function as labels are not one-hot encoded
  # use AdamW optimizer initially with a weight decay of 0.0
  # use the Sparse Categorical Accuracy metric to monitor and save the best model weights
  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                optimizer=tfa.optimizers.AdamW(learning_rate=config.LEARNING_RATE, weight_decay=0.0),
                metrics=tf.keras.metrics.SparseCategoricalAccuracy())

# initialize the best accuracy at - infinity
best_acc = -np.inf
# define number of steps per training epoch
train_steps = (len(train_masked_labels)//config.BATCH_SIZE) + 1
# define number of steps per evaluation epoch
valid_steps = (len(valid_masked_labels)//config.BATCH_SIZE) + 1

# for each epoch iterate over the whole training and validation sets
for epoch in range(config.EPOCHS):
  print(f'Epoch: {epoch+1}/{config.EPOCHS}')
  
  # initialize losses and accuracies list
  losses = []
  accs = []
  # iterate over the training dataset
  for step, (x, y, mask) in enumerate(tqdm(train_ds, total=train_steps)):
    # stop when hitting the number of steps per epoch
    if step == train_steps:
      break
    # train on each batch and pass the sample weights which gives weights to
    # the masked tokens only
    train_results = model.train_on_batch(x=x, y=y, sample_weight=mask,
                                         reset_metrics=True, return_dict=True)
    # append the loss value 
    losses.append(train_results['loss'])
    # append the accuracy value
    accs.append(train_results['sparse_categorical_accuracy'])
    
  print('Train loss:', np.mean(losses), 'Train accuracy', np.mean(accs))
  
  # initialize losses and accuracies list
  losses = []
  accs = []
  # iterate over the validation dataset
  for step, (x, y, mask) in enumerate(tqdm(valid_ds, total=valid_steps)):
    # stop when hitting the number of steps per epoch
    if step == valid_steps:
      break
    # evaluate on each batch and pass the sample weights which gives weights to
    # the masked tokens only
    valid_results = model.test_on_batch(x=x, y=y, sample_weight=mask,
                                         reset_metrics=True, return_dict=True)
    # append the loss value 
    losses.append(valid_results['loss'])
    # append the accuracy value
    accs.append(valid_results['sparse_categorical_accuracy'])
  # monitor the accuracy metric and save model weights whenever there is an improvement
  if np.mean(accs) > best_acc:
    best_acc = np.mean(accs)
    model.save_weights('best_pre_trained_weights.h5')
    print('new model weights saved')
    print('accuracy improved to', best_acc)
  else:
    print("accuracy didn't improve from", best_acc)
  print('Valid loss:', np.mean(losses), 'Valid accuracy', np.mean(accs))

Epoch: 1/100


100%|██████████| 6/6 [00:58<00:00,  9.77s/it]


Train loss: 0.8726251522699991 Train accuracy 0.10984216847767432


100%|██████████| 1/1 [00:12<00:00, 12.69s/it]


new model weights saved
accuracy improved to 0.16273561120033264
Valid loss: 0.8706112504005432 Valid accuracy 0.16273561120033264
Epoch: 2/100


...

accuracy didn't improve from 0.1630197912454605
Valid loss: 0.704608142375946 Valid accuracy 0.10466987639665604
Epoch: 100/100


100%|██████████| 6/6 [00:03<00:00,  1.71it/s]


Train loss: 0.69801464676857 Train accuracy 0.10953203216195107


100%|██████████| 1/1 [00:00<00:00,  1.74it/s]

accuracy didn't improve from 0.1630197912454605
Valid loss: 0.7040518522262573 Valid accuracy 0.10542766749858856
